In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DGX', 'DIS', 'DOCS', 'DOCU', 'EDIT', 'ETSY',
    'FLGT', 'FTNT', 'GH', 'GMED', 'GOOG', 'IDXX', 'ISRG',
    'JD', 'KNSL', 'LRCX', 'LSPD', 'LULU', 'MA', 'MASI', 'MDB',
    'MELI', 'MNST', 'MSFT', 'MTCH', 'NET', 'NOW', 'NVCR', 'NVDA',
    'ODFL', 'OKTA', 'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON',
    'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD', 'ROKU',
    'RVLV', 'SAM', 'SBUX', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC',
    'TEAM', 'TMUS', 'TREX', 'TSCO', 'TSLA', 'TTC', 'TTD', 'TTWO',
    'TWLO', 'TYL', 'UPST', 'VEEV', 'VRTX', 'XPEV',
    'ZBRA', 'ZM']
LINGERERS = [
    'AGI', 'AMEH', 'APPS', 'BRTX', 'BROS', 'BTTR', 'BYRN',
    'CEF', 'CGAU', 'CPRX', 'CTGO', 'DQ', 'DTST', 'DUOL', 'EA', 'EP',
    'EPAM', 'EPSN',  'ESOA', 'FCUV', 'FNV', 'FSI', 'GCBC', 'GME',
    'GNRC', 'GTLS', 'HBM', 'HMY', 'IDR', 'INTG', 'INTZ', 'LPLA',
    'MCVT', 'MICS', 'MNMD', 'MOBQ', 'MTA', 'OMQS', 'PACB', 'PCRX',
    'POWW', 'PRPH', 'QLYS', 'RVP', 'RVYL', 'SAND', 'SPWR', 'STKL',
    'UI', 'VIRT', 'VLD', 'WAVD', 'WKHS', 'WRN', 'XPEV', 'ZEST',
    'ZYXI']
BEST_WEIGHTED =  [
    'AREN', 'BB', 'CHUY', 'CSIQ', 'CUBI', 'DYAI', 'FSM', 'LFMD',
    'LVS', 'OPNT', 'OSTK', 'PBF', 'PFIE', 'SBSW', 'SSTK',
    'URBN', 'VTSI', 'WYNN']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))
len(SYMBOLS)

159

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [10]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ACN
ADBE


### E*Trade

In [11]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [14]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
003CVR016,--
ACN,2216.4
ADYEY,2590.5
AMD,1099
AMGN,23812.47


In [15]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
003CVR016,--
ACN,2216
ADYEY,2590
AMD,1099
AMGN,23812


In [16]:
#et_df.drop(index=['98986CONT'], inplace=True)
et_df.head()

,ET
003CVR016,--
ACN,2216
ADYEY,2590
AMD,1099
AMGN,23812


In [17]:
#et_df.drop('98936J101', inplace=True)

In [18]:
et_df.drop('003CVR016', inplace=True)
et_df.loc[et_df.ET == '--']
#et_df.loc['TYL', 'ET'] = 326.19

,ET


In [19]:
check_symbols(et_df)

All clear


In [20]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,0
ABNB,0
ACN,2216
ADBE,0


### Fidelity

In [21]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Feb-17-2023.csv'

In [22]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips.csv
Portfolio_Positions_Feb-17-2023.csv
Positions.csv
Positions_damiansp_Stocks_2023_02_17.xls


In [23]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-06-", "-07-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,9441.71,$1.00,$0.00,$9441.71,NaN,NaN,NaN,NaN,25.66%,NaN,NaN,Cash
1,218520300,Rollover IRA,003CVR016,ABIOMED INC SHS,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00%,--,--,Cash
2,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.00,$78.50,-$1.58,$1413.00,-$28.44,-1.98%,+$66.09,+4.90%,3.84%,$1346.91,$74.83,Cash
3,218520300,Rollover IRA,AMGN,AMGEN INC,28.00,$240.53,+$6.31,$6734.84,+$176.68,+2.69%,-$199.86,-2.89%,18.30%,$6934.70,$247.67,Cash
4,218520300,Rollover IRA,BROS,DUTCH BROS INC CL A,1.00,$38.56,-$0.80,$38.56,-$0.80,-2.04%,-$1.06,-2.68%,0.10%,$39.62,$39.62,Cash


In [24]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
2,Rollover IRA,AMD,1413
3,Rollover IRA,AMGN,6735
4,Rollover IRA,BROS,39
5,Rollover IRA,BYND,71
6,Rollover IRA,CEF,330


In [25]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
AMD,1413
AMGN,6735
BROS,39
BYND,71
CEF,330


In [26]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [27]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,610,0
ABMD,0,0,0,0
ABNB,0,0,132,0
ACN,2216,0,1662,0
ADBE,0,0,0,0


### TD Ameritrade

In [28]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [29]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

#path = f'{DL}/Positions_damiansp_Stocks_2022_11_04.xls'

In [30]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1].replace(',', ''))
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [31]:
td = parse_file(path)
td.head()

,TD
ADBE,356.85
AMGN,962.12
AMZN,194.00
APPS,366.27
AREN,9.57


In [32]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,0,0,610,0,0
ABMD,0,0,0,0,0
ABNB,0,0,132,0,0
ACN,2216,0,1662,0,0
ADBE,0,0,0,0,357


In [33]:
rsi = pd.read_csv(
    '~/Downloads/Holdings - Damian Satterthwaite-Phillips.csv',
    index_col=0,
    usecols=['Symbol', 'Value'])
rsi.Value = rsi.Value.apply(
    lambda x: round(float(x.replace('$', '').replace(',', ''))))
out = pd.concat([out, rsi], axis=1)
out.rename(columns={'Value': 'RSI'}, inplace=True)
out['RSI'] = out.RSI.fillna(0).astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD,RSI
AAPL,0.0,0.0,610.0,0.0,0.0,305
ABMD,0.0,0.0,0.0,0.0,0.0,0
ABNB,0.0,0.0,132.0,0.0,0.0,0
ACN,2216.0,0.0,1662.0,0.0,0.0,831
ADBE,0.0,0.0,0.0,0.0,357.0,714


In [34]:
adel = pd.read_csv(
    '~/Downloads/Holdings - Damian Satterthwaite-Phillips(1).csv',
    index_col=0,
    usecols=['Symbol', 'Value'])
adel.Value = adel.Value.apply(
    lambda x: round(float(x.replace('$', '').replace(',', ''))))
out = pd.concat([out, adel], axis=1)
out.rename(columns={'Value': 'Adel'}, inplace=True)
out['Adel'] = out.Adel.fillna(0).astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD,RSI,Adel
AAPL,0.0,0.0,610.0,0.0,0.0,305.0,610
ABMD,0.0,0.0,0.0,0.0,0.0,0.0,0
ABNB,0.0,0.0,132.0,0.0,0.0,0.0,0
ACN,2216.0,0.0,1662.0,0.0,0.0,831.0,554
ADBE,0.0,0.0,0.0,0.0,357.0,714.0,1071


In [35]:
out.to_csv(f'{DL}/clean.csv')

In [36]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*